## final_analysis

Here we combine all the earlier defined models and tools and use them for the final task of word-density-analysis

Setting things up

In [1]:
import sys
sys.path.append('../modules/')
import nlp
from webpage import *

In [3]:
webpages ={ website:Webpage(test_url[website]) for website in test_url}
[page.load() for page in webpages.values()]

c:\anaconda\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
../modules/webpage.py:93: UserWarning: Unable to correctly parse given URL:http://www.nytimes.com/2015/09/11/world/netanyahu-makes-quick-pivot-from-loss-on-iran-deal.html
  warn('Unable to correctly parse given URL:%s' %self.url)


[None, None, None, None, None, None, None, None]

In [8]:
dir(webpages.values()[0])

['__doc__',
 '__goose_article__',
 '__init__',
 '__module__',
 '_valid_text_',
 'html',
 'load',
 'meta_description',
 'meta_keywords',
 'set_url',
 'tags',
 'text',
 'title',
 'url']

## For each webpage

0. NNP from Pos tags of title,description
1. Return a list of keywords
2. Return the ngrams
3. Return the entities 

In [244]:
[w.tags for w in webpages.values()]

[[],
 [],
 [],
 [],
 [],
 [u'Life Stages', u'Hike', u'Jennifer Davis-Flynn', u'camp', u'Tips'],
 [],
 []]

In [9]:
[w.meta_keywords for w in webpages.values()]

['',
 'reddit, reddit.com, vote, comment, submit',
 '',
 u'Latest,Atletico Madrid,Spanish Primera Divisi\xf3n,Barcelona,XBOX 360',
 u'',
 '',
 'toaster',
 'politics, Man behind NSA leaks says he did it to safeguard privacy, liberty - CNNPolitics.com']

In [23]:
[[tag for tag,token in nlp.reduced_pos_tag(w.title) if token == 'NNP'] for w in webpages.values()]

[['Georgia Institute of Technology College of Computing', 'Wikipedia'],
 ['Barcelona News', 'Discussion /r/Barca'],
 ['Louis van Gaal', 'Manchester United', 'Marcos Rojo'],
 ['Barca', 'Atleti'],
 [],
 ['Introduce Your Indoorsy Friend', 'Outdoors'],
 ['Cuisinart CPT-122 Compact 2-Slice Toaster', 'Kitchen', 'Dining'],
 ['NSA']]

In [246]:
def get_meta_info(w):
    """ Extract relevant information from the website metadata"""
    
    # List of proper nouns from the title
    title_nouns = [tag for tag,token in nlp.reduced_pos_tag(w.title) if token == 'NNP']
    
    # List of processed meta_keywords
    keywords = nlp.pre_process(w.meta_keywords)
    
    # List of procesed tags
    tags = nlp.pre_process(' '.join(w.tags))
    
    return title_nouns,keywords,tags

In [250]:
get_meta_info(webpages['goal'])

(['Louis van Gaal', 'Manchester United', 'Marcos Rojo'], [], [])

In [12]:
[nlp.ngram(w.text) for w in webpages.values()]

[({('1964',): 2,
   ('1988',): 2,
   ('2006',): 3,
   ('2010',): 2,
   ('2013',): 2,
   ('also',): 4,
   ('announced',): 2,
   (u'art',): 2,
   (u'award',): 2,
   ('began',): 4,
   ('building',): 2,
   ('college',): 23,
   ('computational',): 2,
   ('computer',): 15,
   ('computing',): 16,
   (u'concept',): 2,
   ('contribution',): 2,
   ('dean',): 3,
   ('degree',): 9,
   ('demillo',): 2,
   (u'department',): 2,
   ('director',): 2,
   (u'division',): 3,
   ('engineering',): 2,
   ('example',): 2,
   ('expanded',): 2,
   ('faculty',): 3,
   ('first',): 6,
   ('following',): 2,
   ('formalized',): 2,
   ('georgia',): 13,
   ('graduate',): 2,
   ('group',): 3,
   ('hired',): 2,
   ('history',): 2,
   ('include',): 3,
   ('including',): 2,
   ('information',): 8,
   ('institute',): 2,
   ('interactive',): 2,
   ('john',): 2,
   ('klaus',): 2,
   ('liberal',): 2,
   (u'master',): 3,
   (u'medium',): 2,
   (u'member',): 2,
   ('miller',): 2,
   ('offer',): 3,
   ('offered',): 4,
   ('offer

In [13]:
[nlp.entity_count(w.text) for w in webpages.values()]

[{u'LOCATION': {u'United States': 4},
  u'ORGANIZATION': {u'Carnegie Mellon University': 1,
   u'College of Computing Dean': 1,
   u'College of Engineering and College of Sciences': 1,
   u'Computational Media': 1,
   u'Computational Science & Engineering': 2,
   u'Computer Science': 1,
   u'DARPA': 1,
   u'Emory University': 1,
   u'Georgia Institute of Technology': 1,
   u'Georgia Tech': 11,
   u"Georgia Tech 's College of Computing": 1,
   u"Georgia Tech 's School of Literature": 1,
   u'IBM': 1,
   u'Ivan Allen College of Liberal Arts': 1,
   u'School of Computer Science': 3,
   u'School of Information Science': 1,
   u'School of Information and Computer Science': 3,
   u'School of Interactive Computing': 1,
   u'The College of Computing': 7},
  u'PERSON': {u'Barack Obama': 1,
   u'Chris Klaus': 2,
   u'Gary Schuster': 1,
   u'James D. Foley': 1,
   u'John Patrick Crecine': 2,
   u'Nordiar Waldemar Ziegler': 1,
   u'Peter A. Freeman': 1,
   u'Ray Miller': 2,
   u'Richard DeMillo': 

In [163]:
tagged_entity_count = nlp.entity_count(webpages['reddit'].text) 

In [164]:
entity_count = {entity:tagged_entity_count[tag][entity]  for tag in tagged_entity_count for entity in tagged_entity_count[tag] }

Top 22% of all entities and they should appear more than once

In [230]:
def get_entities(text):
    """ Return the top 22% entities sorted by frequency """
    tagged_entity_count = nlp.entity_count(text) 
    entity_count = {entity:tagged_entity_count[tag][entity]
                    for tag in tagged_entity_count
                        for entity in tagged_entity_count[tag] }
    return [entity 
                for entity,count in sorted(entity_count.items(),key = lambda x:x[1], reverse = True)[:int(len(entity_count)/4.5)]
            if count > 2 ]

In [241]:
get_entities(webpages['cnn'].text)

[u'Edward Snowden', u'United States', u'Hawaii', u'NSA']

In [256]:
def get_ngrams(text,k1=10,k2=5,k3=2):
    """ Obtain relevant ngrams for the entered text """
    unigrams,bigrams,trigrams = nlp.ngram(text)

    
    relevant_unigrams = [unigram[0] for unigram,count in sorted(unigrams.items(), key = lambda x:x[1], reverse = True) 
                                 ][:len(unigrams)/k1]
    relevant_bigrams = [' '.join(bigram) for bigram,count in sorted(bigrams.items(), key = lambda x:x[1], reverse = True) 
                                 ][:len(bigrams)/k2]
    relevant_trigrams = [' '.join(trigram) for trigram,count in sorted(trigrams.items(), key = lambda x:x[1], reverse = True) 
                                 ][:len(trigrams)/k3]
    
    relevant_trigrams.extend(relevant_bigrams)
    relevant_trigrams.extend(relevant_unigrams)
    
    return relevant_trigrams

In [257]:
get_ngrams(webpages['wiki'].text)

['information computer science',
 'school information computer',
 u'college georgia tech',
 u'college computing school',
 'school interactive computing',
 'college computing',
 'computer science',
 'georgia tech',
 'school information',
 'information science',
 'college',
 'science',
 'school',
 'computing',
 'computer',
 'georgia',
 'tech']

## Finalizing everything

In [251]:
def validate(url):
    """ Check if it is a valid url or not"""
    pass

In [265]:
def word_density_analyze(url):
    
    validate(url)
    w = Webpage(url)
    print "Loading the WebPage"
    w.load()
    print "Loading the WebPage !! COMPLETED !!"
    
    
    # First the meta    
    print "Extracting Meta Data"
    title_nouns,keywords,tags = get_meta_info(w)
    print "Extracting Meta Data !! COMPLETED !!"
    
    # Then the ner tags
    print "Extracting Entities (this may take a while)"
    entities = get_entities(w.text)
    print "Extracting Entities (this may take a while) !! COMPLETED !!"
    
    # Then the ngram counts
    print "Extracting ngram counts (this should be faster than Entities)"
    relevant_ngrams = get_ngrams(w.text)
    print "Extracting ngram counts (this should be faster than Entities) !! COMPLETED !!"
    
    
    print 'Generating Result'
    result = []
    keywords_used = False
    tags_used = False
    ngrams_used = False
    result.extend(title_nouns)
    
    if len(keywords) <= 2:
        result.extend(keywords)
        keywords_used = True
        
    if len(tags) <= 2:
        result.extend(tags)
        tags_used = True
        
    result.extend(entities)    
    
    if len(keywords) > 5 or len(tags) > 3:
        result.extend(relevant_ngrams)
        
    if not keywords_used:
        result.extend(keywords)
    
    if not tags_used:
        result.extend(tags)
        
    if not ngrams_used:
        result.extend(relevant_ngrams)
    
    
    return result
    

In [267]:
print '\n'.join(word_density_analyze( "http://www.amazon.com/Cuisinart-CPT-122-Compact-2-Slice-Toaster/dp/B009GQ034C/ref=sr_1_1?s=kitchen&ie=UTF8&qid=1431620315&sr=1-1&keywords=toaster"))

Loading the WebPage
Loading the WebPage !! COMPLETED !!
Extracting Meta Data
Extracting Meta Data !! COMPLETED !!
Extracting Entities (this may take a while)
Extracting Entities (this may take a while) !! COMPLETED !!
Extracting ngram counts (this should be faster than Entities)
Extracting ngram counts (this should be faster than Entities) !! COMPLETED !!
Generating Result
Cuisinart CPT-122 Compact 2-Slice Toaster
Kitchen
Dining
toast
